Multi Layer Perceptron Classifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import shuffle

df = pd.read_csv('african_crises.csv')
df = shuffle(df, random_state=11)
df.reset_index(inplace=True, drop=True)
# converting into useful numbers

df['banking_crisis'] = df['banking_crisis'].replace('crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(1)
df['banking_crisis'] = df['banking_crisis'].replace('no_crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(0)

# removing unneccesary data

df.drop(['cc3','country'], axis=1, inplace=True)

# scaling the data

df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled['banking_crisis'] = df['banking_crisis']
df = df_scaled

# defining the input data, X, and the desired results, y

X = df.loc[:,df.columns != 'banking_crisis']
y = df.loc[:, 'banking_crisis']

# breaking data into training data, validation data, and test data

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.2,shuffle=False)

# constructing a Multilayer Perceptron

model = Sequential()
model.add(Dense(32,activation = 'relu', input_dim = 11))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy']) # adam, rmsprop, sgd

model.summary()
# training the network

model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)

# scoring it on the data it trained on as well as test data

scores = model.evaluate(X_train, y_train)
print ("Training Accuracy: %.2f%%\n" % (scores[1]*100))

scores = model.evaluate(X_test, y_test)
print ("Testing Accuracy: %.2f%%\n" % (scores[1]*100))


Multi Layer Perceptron Regressor

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# load dataset

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# create model
model = Sequential()
model.add(Dense(8, input_shape=[X_train.shape[1]], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.summary()

model.compile(loss='mean_absolute_error', optimizer='rmsprop', metrics=['mae'])


history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, verbose=1)

print(model.evaluate(X_test_scaled, y_test))


# summarize history for accuracy
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('model mean absolute error')
plt.ylabel('mean absolute error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



Radial Basis Function Classifier

In [ ]:
from keras.layers import Dense, Input
from keras.models import Sequential, Model
from keras.losses import binary_crossentropy
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from keras.layers import Layer
from keras import backend as K

class RBF(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBF, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBF, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)


df = pd.read_csv('african_crises.csv')
df = shuffle(df, random_state=11)
df.reset_index(inplace=True, drop=True)
# converting into useful numbers

df['banking_crisis'] = df['banking_crisis'].replace('crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(1)
df['banking_crisis'] = df['banking_crisis'].replace('no_crisis',np.nan)
df['banking_crisis'] = df['banking_crisis'].fillna(0)

# removing unneccesary data

df.drop(['cc3','country'], axis=1, inplace=True)

# scaling the data

df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled['banking_crisis'] = df['banking_crisis']
df = df_scaled

# defining the input data, X, and the desired results, y

X = df.loc[:,df.columns != 'banking_crisis']
y = df.loc[:, 'banking_crisis']

# breaking data into training data, validation data, and test data

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,shuffle=False)


model = Sequential()
input = Input(shape=(X_train.shape[1],))
rbf=RBF(10, 0.5)(input)
out=Dense(1, activation='sigmoid')(rbf)

model = Model(inputs=input, outputs=out)

model.compile(optimizer='rmsprop', loss=binary_crossentropy, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=256, epochs=30)

scores = model.evaluate(X_train, y_train)
print ("Training Accuracy: %.2f%%\n" % (scores[1]*100))
scores = model.evaluate(X_test, y_test)
print ("Testing Accuracy: %.2f%%\n" % (scores[1]*100))

Convolutional Neural Network - LeNet5

In [ ]:
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

# Instantiate an empty model
model = Sequential()

# C1 Convolutional Layer
model.add(Conv2D(filters=6, kernel_size=5, strides=1, activation='relu', input_shape=(32, 32, 1), padding="valid"))

# S1 Pooling Layer
model.add(MaxPooling2D(pool_size=2, strides=2))

# C2 Convolutional Layer
model.add(Conv2D(filters=16, kernel_size=5, strides=1, activation='relu', input_shape=(14, 14, 6), padding="valid"))

# S2 Pooling Layer
model.add(MaxPooling2D(pool_size=2, strides=2))

# Flatten the output in order to feed the fully connected layers
model.add(Flatten())

# FC1 Fully Connected Layer
model.add(Dense(units=120, activation='relu'))

# FC2 Fully Connected Layer
model.add(Dense(units=84, activation='relu'))

# FC3 Fully Connected Layer
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Convolutional Neural Network - AlexNet

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()


# 1st Convolutional Layer
model.add(Conv2D(filters=96, activation='relu',input_shape=(224, 224, 3), kernel_size=(11, 11), strides=(4, 4)))
# Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, activation='relu', kernel_size=(11, 11), strides=(1, 1)))
# Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, activation='relu', kernel_size=(3, 3), strides=(1, 1)))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, activation='relu', kernel_size=(3, 3), strides=(1, 1)))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, activation='relu', kernel_size=(3, 3), strides=(1, 1)))
# Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())


# 1st Dense Layer
model.add(Dense(4096, activation='relu', input_shape=(224*224*3,)))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096, activation='relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000, activation='relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(17, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Pretrained networks - InceptionV3


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

model = InceptionV3(input_tensor=Input(shape=(224, 224, 3)) , weights='imagenet', include_top=True)

model.summary()